# Temperature near Rome

## Import libraries

In [ ]:
import warnings

import xarray as xr
from zarr.errors import ZarrUserWarning

xr.set_options(use_new_combine_kwarg_defaults=True)
warnings.simplefilter("ignore", category=ZarrUserWarning)

## Set parameters

In [ ]:
URL = "https://data.earthdatahub.destine.eu"
COLLECTION = "cmip6"
ITEMS = [
    "CMCC-CM2-SR5-historical-r1i1p1f1-day-gn",
    "CMCC-CM2-SR5-ScenarioMIP-r1i1p1f1-day-gn",
    "DKRZ-MPI-ESM1-2-HR-historical-r1i1p1f1-day-gn",
    "DKRZ-MPI-ESM1-2-HR-ScenarioMIP-r1i1p1f1-day-gn",
    "EC-Earth3-CC-historical-r1i1p1f1-day-gr",
    "EC-Earth3-CC-ScenarioMIP-r1i1p1f1-day-gr",
    "IPSL-CM6A-LR-historical-r1i1p1f1-day-gr",
    "IPSL-CM6A-LR-ScenarioMIP-r1i1p1f1-day-gr",
    "NCAR-CESM2-historical-r1i1p1f1-day-gn",
    "NCAR-CESM2-ScenarioMIP-r10i1p1f1-day-gn",
    "NCC-NorESM2-MM-historical-r1i1p1f1-day-gn",
    "NCC-NorESM2-MM-ScenarioMIP-r1i1p1f1-day-gn",
]
VERSION = 0
VARIABLE = "tas"
SELECTION = {"lat": 41.8967, "lon": 12.4822}

## Open datasets, select variable, compute annual mean, and combine

In [ ]:
dataarrays = []
for item in ITEMS:
    print(f"Opening {item=}")
    ds = xr.open_dataset(
        "/".join([URL, COLLECTION, f"{item}-v{VERSION}.zarr"]),
        storage_options={"client_kwargs": {"trust_env": True}},
        chunks={},
        engine="zarr",
    )
    da = ds[VARIABLE].sel(**SELECTION, method="nearest")
    da = da.groupby("time.year").mean()
    da = da.expand_dims(model=item.rsplit("-", 4)[:1])
    if "experiment_id" not in da.dims:
        da = da.sel(year=slice(None, 2014))
        da = da.expand_dims(experiment_id=["historical"])
    dataarrays.append(da.chunk(year=-1).reset_coords(drop=True))
ds = xr.combine_by_coords(dataarrays, combine_attrs="drop_conflicts", join="outer")
da = ds[VARIABLE]

## Load annual temperatures in degree Celsius

In [ ]:
with xr.set_options(keep_attrs=True):
    da = (da - 273.15).compute()
da.attrs["units"] = "°C"

## Plot

In [ ]:
facet = da.drop_vars("experiment_id").plot(
    col="model", col_wrap=2, cmap="RdBu_r", aspect=2
)
for ax in facet.axs.flatten():
    ax.grid()
    ax.set_yticks(range(da.sizes["experiment_id"]), da["experiment_id"].values.tolist())